# gpx2gcode

How to 3D print your rides with non-planar 3D printing

## Prerequisites
Create a python virtual environment
```
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```

In [ ]:
import numpy as np
import gpxpy
import pyproj
import folium
from branca.colormap import linear
import fullcontrol as fc

## Getting a GPX of your ride
You can download this from the map on an activity page on Strava, or from a Garmin Connect connect activity from the settings icon.

Change the gpx_path variable to match the relative location of the gpx file.

I've found that activities use the recording device's barometric sensor data to estimate the altitude and this can sometimes be wrong. In garmin connect, you can save an activity as a course and then it will re-calculate the altitude based on map data rather than activity data.


In [ ]:
gpx_path = "example_rides/ubes_bikepacking_2.0.gpx"

## Open a gpx file
lat_long_points= np.empty((0, 2))
elevation = np.empty((0, 1))

# Parsing a GPX file
with open(gpx_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            lat_long_points = np.vstack((lat_long_points, np.array([point.latitude, point.longitude])))
            elevation = np.append(elevation, point.elevation)

In [ ]:
## Calculate zone from lat/long

centre = np.min(lat_long_points, axis=0) + (np.ptp(lat_long_points, axis=0) / 2)

zone = int((centre[1] + 180) / 6) + 1

P = pyproj.Proj(proj='utm', zone=zone, ellps='WGS84', preserve_units=True)
G = pyproj.Geod(ellps='WGS84')

utm_points = np.empty((0, 2))
for point in lat_long_points:
    utm_points = np.vstack((utm_points, P(point[1], point[0])))

lat_long_centre = np.min(lat_long_points, axis=0) + (np.ptp(lat_long_points, axis=0) / 2)


In [ ]:
## Create a new list of points that contains one point for every 10m of the points (smoothing things out to avoid scribbles where stationary)
distance_sum = 0
point_couter = 0
utm_position_sum = np.array([0,0], dtype=float)
lat_long_position_sum = np.array([0,0], dtype=float)
last_pos = utm_points[0].astype(np.float64)

lat_long_points_smooth = np.array([lat_long_points[0]], dtype=float)
utm_points_smooth = np.array([utm_points[0]], dtype=float)
new_elevation = np.array([elevation[0]], dtype=float)

for i in range(1, len(utm_points)):
    distance = np.sqrt(np.sum((utm_points[i].astype(np.float64) - last_pos)**2))
    utm_position_sum += utm_points[i].astype(np.float64)  # Explicitly convert position_sum to float64
    lat_long_position_sum += lat_long_points[i].astype(np.float64)  # Explicitly convert position_sum to float64
    point_couter += 1
    if distance >= 250:
        utm_points_smooth = np.vstack((utm_points_smooth, utm_position_sum/point_couter))
        lat_long_points_smooth = np.vstack((lat_long_points_smooth, lat_long_position_sum/point_couter))
        point_couter = 0
        last_pos = utm_points[i].astype(np.float64)
        utm_position_sum = np.array([0,0], dtype=float)
        lat_long_position_sum = np.array([0,0], dtype=float)
        new_elevation = np.append(new_elevation, elevation[i])




In [ ]:
## When elevation is none, just use the last
for i in range(1, len(new_elevation)):
    if new_elevation[i] is None:
        new_elevation[i] = new_elevation[i-1]




In [ ]:
## Create a colormap based on altitude values
altitude_colormap = linear.Spectral_11.scale(new_elevation.min(), new_elevation.max())

colours = list(map(altitude_colormap, new_elevation))

## Create a function to assign color based on altitude
def color_function(altitude):
    return altitude_colormap(altitude)

## Show map using utm coordinates
m = folium.Map(location=[lat_long_centre[0], lat_long_centre[1]], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(m)

## Add overlay of lines onto map with color based on altitude
for i in range(1, len(lat_long_points_smooth)):
    folium.PolyLine([lat_long_points_smooth[i-1], lat_long_points_smooth[i]], color=color_function(new_elevation[i-1]), weight=2.5).add_to(m)


## Add colorbar to the map
altitude_colormap.caption = 'Altitude'
m.add_child(altitude_colormap)

## Generate Toolpaths
Here we generate the toolpaths for the 33D printer. We're using the "generic" printer profile from fullcontrol, you may want to change this to something else, or define your own.

We also are using a max layer height of 0.4mm and layer width of 1mm. This works well with a .8mm nozzle, but fiddle with these variables for best results

In [ ]:
#print the size of the bounding box of utm_points
print(np.min(utm_points, axis=0))
print(np.max(utm_points, axis=0))
print(np.ptp(utm_points, axis=0))

In [ ]:

gcode = []

coordinate_scale = 1/1500000

altitude_scale = 1/250000
printed_altitude_range = np.ptp(new_elevation)*1000*altitude_scale

layer_count = 15
layer_height = 0.4

layer_height_range = printed_altitude_range/layer_count
min_layer_height = layer_height-layer_height_range

extrusion_multiplier = 0.95


smoothing = 33
wrap = smoothing//2

# apply some kind of smoothing to the altitude
smooth_elevation = np.concatenate((new_elevation[:-wrap], new_elevation, new_elevation[:wrap]))
smooth_elevation = np.convolve(smooth_elevation, np.ones(smoothing)/smoothing, mode='valid')


## Total height  = some base amount + some factor of the altitude



scaled_points = utm_points_smooth*coordinate_scale*1000 # convert to mm
scaled_points -= np.min(scaled_points, axis=0)
scaled_points += np.array([10,10])
constant_factor = min_layer_height/layer_height
elevation_factor = ((smooth_elevation-np.min(smooth_elevation))/(np.ptp(smooth_elevation))*(1-constant_factor))+constant_factor

gcode.append(fc.ManualGcode(text='M601'))



for i in range(layer_count):
    for j, point in enumerate(scaled_points):
        gcode.append(fc.ExtrusionGeometry(area_model='stadium', width = 1*extrusion_multiplier, height = 0.4*elevation_factor[j]))
        gcode.append(fc.Point(x=point[0], y=point[1],z=0.6+(0.4*i)*elevation_factor[j]))
    
gcode_controls = fc.GcodeControls(printer_name='prusa_mini', initialization_data={'print_speed': 1200, 'primer': 'front_lines_then_y',}, ) 
controls=fc.PlotControls(style='tube')

fc.transform(gcode, 'plot', controls)

## Export the GCODE
Change `out.gcode` to what you want to save the GCODE file as.

In [ ]:
lines = fc.transform(gcode, 'gcode', gcode_controls)
with open('out.gcode', 'w') as f:
    f.write(lines)